In [ ]:
# we need the following libraries, so let's install them
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import matplotlib.pyplot as plt
import sys
import warnings
from sklearn.utils import shuffle
import random
import logging
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# loadnig the data and removing the useless column 'Unnamed: 0'
# whole_data = pd.read_csv('/content/gdrive/MyDrive/radiomics/Chexpert_synced_data_nan_free.csv')
whole_data = pd.read_csv('Chexpert_synced_data_nan_free.csv')
whole_data.head(5)


,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,-669.603949,685.781964,1.059520e+10,6.017600,866.092155,1.713989,934.117676,428.763596,10.279660,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,-801.474548,564.486328,1.274714e+10,6.146630,954.811813,1.756523,1020.768616,468.058052,-132.958831,...,0.0,1.0,1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0
2,2,-844.417249,307.704349,9.364581e+09,5.848908,690.788330,1.913630,589.552979,354.572276,-308.642302,...,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
3,4,-742.036096,673.796082,1.132259e+10,6.076017,1038.149048,1.627926,933.427429,467.972161,-17.909900,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,5,-802.322357,560.980408,1.410547e+10,6.061123,783.288322,2.001557,869.513733,412.838385,-210.721353,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0


In [ ]:
2# excludiung Ages with Nans
whole_data = whole_data[~pd.isnull(whole_data['Sex'])]
whole_data.shape

(188421, 489)

In [ ]:
new_target = []
for string_target in whole_data['Sex'].values:
  if 'Male' in string_target:
    new_target.append(1)
  elif 'Female' in string_target:
    new_target.append(0)
  else:
    new_target.append(np.nan)
whole_data['New_sex']= new_target

In [ ]:
whole_data = whole_data[~pd.isnull(whole_data['New_sex'])]
whole_data.head()

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,New_sex
0,0,-669.603949,685.781964,1.059520e+10,6.017600,866.092155,1.713989,934.117676,428.763596,10.279660,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,-801.474548,564.486328,1.274714e+10,6.146630,954.811813,1.756523,1020.768616,468.058052,-132.958831,...,1.0,1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,2,-844.417249,307.704349,9.364581e+09,5.848908,690.788330,1.913630,589.552979,354.572276,-308.642302,...,1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,-742.036096,673.796082,1.132259e+10,6.076017,1038.149048,1.627926,933.427429,467.972161,-17.909900,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,5,-802.322357,560.980408,1.410547e+10,6.061123,783.288322,2.001557,869.513733,412.838385,-210.721353,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0


In [ ]:
random_seed = 21202
del whole_data['Unnamed: 0']
whole_data.head()
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)

In [ ]:
# finding total number of samples:
n_samples = whole_data.shape[0]
print("number of samples are: {}".format(n_samples))

number of samples are: 188420


In [ ]:
# names of all targets:
targets_names = list(whole_data.iloc[:,464:])
print(targets_names)

['index', 'Path', 'Patient_id', 'Study_Number', 'Sex', 'Age', 'Race', 'Ethnicity', 'Frontal/Lateral', 'AP/PA', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices', 'New_sex']


In [ ]:
# finding the number of unique subjects:
unique_subjects = pd.unique(whole_data['Patient_id'])
print('The number of unique subjects are: {0}'.format(len(unique_subjects)))

The number of unique subjects are: 64484


In [ ]:
# preallocating the frequency of session of the subjects:
# this is written to make sure that we dont use a subject in both training and testing split.
test_split = 0.06       # precent out of total samples
valid_split = 0.1       # precent out of total samples (this is the sumation of both training and validation splits)

n_test = n_samples * test_split
n_whole_train = n_samples - n_test

frequencies = np.ones((unique_subjects.shape[0], 4)) * np.nan
for i, sbj in tqdm(enumerate(unique_subjects)):
  frequencies[i, 0] = sbj
  frequencies[i, 1] = (whole_data['Patient_id'] == sbj).sum()
cumolative_sums_of_samples = np.cumsum(frequencies[:,1])
frequencies[:, 2] = cumolative_sums_of_samples
frequencies[:, 3] = cumolative_sums_of_samples/cumolative_sums_of_samples[~0]

64484it [00:17, 3754.96it/s]


In [ ]:
# visualizing the frequency  of subjects and images taken from each
vis_frequencies = pd.DataFrame(frequencies,columns=['subject_ID','n_of_sessions/images','cumolative_sessions','preccent_of_number_of_images'])
vis_frequencies.head(15)

,subject_ID,n_of_sessions/images,cumolative_sessions,preccent_of_number_of_images
0,34504.0,5.0,5.0,0.000027
1,59349.0,2.0,7.0,0.000037
2,38301.0,38.0,45.0,0.000239
3,17270.0,1.0,46.0,0.000244
4,37343.0,3.0,49.0,0.000260
5,35867.0,3.0,52.0,0.000276
6,28124.0,1.0,53.0,0.000281
7,17012.0,13.0,66.0,0.000350
8,38999.0,5.0,71.0,0.000377
9,7951.0,5.0,76.0,0.000403


In [ ]:
# separating the testing split without having any mutual_subjects:
testing_subjects_masking = vis_frequencies['preccent_of_number_of_images'] < test_split
testing_subjects = vis_frequencies[testing_subjects_masking]['subject_ID'].values.tolist()
whole_train_subjects = vis_frequencies[~testing_subjects_masking]['subject_ID'].values.tolist()

In [ ]:
# excluding irrelevant features
excluding2ds =  np.concatenate( (np.arange(32,41,dtype=int),
                                 np.arange(32,41,dtype=int)+ 116,
                                 np.arange(32,41,dtype=int)+116*2,
                                 np.arange(32,41,dtype=int)+116*3))

whole_data.drop(whole_data.columns[excluding2ds.tolist()],axis = 1,inplace=True)

In [ ]:
# separating test_data and whole_trainig_data
Test = whole_data[whole_data['Patient_id'].isin(testing_subjects)]
X_test, y_test = Test.iloc[:,:464-36].values, Test['New_sex'].values

# separating test_data and whole_trainig_data
Train = whole_data[whole_data['Patient_id'].isin(whole_train_subjects)]

# the size of training/testing_split:
print('The size of the testing split is: {}'.format(Test.shape))
print('The size of the whole trainig (train + valid) split is: {}'.format(Train.shape))

The size of the testing split is: (11284, 453)
The size of the whole trainig (train + valid) split is: (177136, 453)


In [ ]:
# splitting the validation split:
X_train, X_val, y_train, y_val = train_test_split(Train.iloc[:,:464-36].values,
                                                  Train['New_sex'].values,
                                                  test_size=valid_split,
                                                  random_state=random_seed
                                                  )

In [ ]:
# checking the number of nans:
pd.isnull(X_train).sum(), pd.isnull(X_test).sum(), pd.isnull(X_val).sum()

(0, 0, 0)

In [ ]:
print('The size of the validation split is: {}'.format(X_val.shape))
X_train.shape, X_val.shape, y_train.shape, y_val.shape

The size of the validation split is: (17714, 428)


((159422, 428), (17714, 428), (159422,), (17714,))

In [ ]:
def classifer_1(features_number: int) -> callable:
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=100, activation="tanh", input_dim=features_number))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=100, activation="tanh"))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    model.summary()
    return model

def compiler_1(X_train_, Y_train_, X_valid_, Y_valid_,mdl_callback):
  initial_learning_rate = 0.001
  epochs = 500
  batch_size = 256
  model = classifer_1(features_number=X_train_.shape[1])
  lss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  metrics = ['accuracy',
                        tf.keras.metrics.AUC(curve='PR',name='PR-curve'),
                        tf.keras.metrics.AUC(curve='ROC',name='ROC-curve')]
  # metrics = metrics=[tf.keras.metrics.RootMeanSquaredError()]
  # metrics = [tf.keras.metrics.RootMeanSquaredError()]
  # lss = 'mean_absolute_error'
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-08),
                                                loss=lss,
                metrics=metrics)
  model.fit(x=X_train_,
            y=Y_train_,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_valid_, Y_valid_),
            verbose=2,
            shuffle=True,
            callbacks=[mdl_callback])
  return model

In [ ]:
# full Model

X_train_ = X_train.copy()
X_val_ = X_val.copy()
X_test_ = X_test.copy()

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + 'full'+ '_'
saving_path = 'chexpert_sex_' + 'full'+ '_'

belongings={'mean':mean_val,
            'feature_number':None,
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               42100     
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 52301 (204.30 KB)
Trainable params: 52301 (204.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

/Users/mra/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.42649 to 0.41034, saving model to chexpert_sex_full__mdl.h5
623/623 - 2s - loss: 0.4138 - accuracy: 0.8092 - PR-curve: 0.9166 - ROC-curve: 0.8882 - val_loss: 0.4103 - val_accuracy: 0.8115 - val_PR-curve: 0.9159 - val_ROC-curve: 0.8921 - 2s/epoch - 2ms/step
Epoch 3/500

Epoch 3: val_loss improved from 0.41034 to 0.40725, saving model to chexpert_sex_full__mdl.h5
623/623 - 2s - loss: 0.3987 - accuracy: 0.8181 - PR-curve: 0.9235 - ROC-curve: 0.8968 - val_loss: 0.4073 - val_accuracy: 0.8124 - val_PR-curve: 0.9174 - val_ROC-curve: 0.8931 - 2s/epoch - 2ms/step
Epoch 4/500

Epoch 4: val_loss improved from 0.40725 to 0.39570, saving model to chexpert_sex_full__mdl.h5
623/623 - 2s - loss: 0.3885 - accuracy: 0.8237 - PR-curve: 0.9278 - ROC-curve: 0.9025 - val_loss: 0.3957 - val_accuracy: 0.8196 - val_PR-curve: 0.9232 - val_ROC-curve: 0.8994 - 2s/epoch - 3ms/step
Epoch 5/500

Epoch 5: val_loss improved from 0.39570 to 0.38691, saving model to chexpert_sex_full__

In [ ]:
# each segments:

segment_number = 0

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_sex_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}
np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               10600     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_13 (Dense)            (None, 100)               10100     
                                                                 
 dropout_9 (Dropout)         (None, 100)               0         
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 20801 (81.25 KB)
Trainable params: 20801 (81.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epoch

In [ ]:
# each segments:
segment_number = 1

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_sex_segment_' + segment_names[segment_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'
# checkpoint_filepath = '_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 100)               10600     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_17 (Dense)            (None, 1)                 101       
                                                                 
Total params: 20801 (81.25 KB)
Trainable params: 20801 (81.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epoch

In [ ]:
# each segments:
segment_number = 2

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_sex_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 100)               10600     
                                                                 
 dropout_12 (Dropout)        (None, 100)               0         
                                                                 
 dense_19 (Dense)            (None, 100)               10100     
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_20 (Dense)            (None, 1)                 101       
                                                                 
Total params: 20801 (81.25 KB)
Trainable params: 20801 (81.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epoch

In [ ]:
# each segments:
segment_number = 3

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'

saving_path = 'chexpert_sex_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               10600     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dropout_15 (Dropout)        (None, 100)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 101       
                                                                 
Total params: 20801 (81.25 KB)
Trainable params: 20801 (81.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epoch

In [ ]:
fos_features = (np.arange(0,18) + 0).tolist() + (np.arange(0,18) + 107).tolist() + (np.arange(0,18) + 107*2).tolist() + (np.arange(0,18) + 107*3).tolist()
_3d_features = (np.arange(18,32) + 0).tolist() + (np.arange(18,32) + 107).tolist() + (np.arange(18,32) + 107*2).tolist() + (np.arange(18,32) + 107*3).tolist()
glcm_features = (np.arange(32,56) + 0).tolist() + (np.arange(32,56) + 107).tolist() + (np.arange(32,56) + 107*2).tolist() + (np.arange(32,56) + 107*3).tolist()
glszm_features = (np.arange(56,72) + 0).tolist() + (np.arange(56,72)+ 107).tolist() + (np.arange(56,72) + 107*2).tolist() + (np.arange(56,72) + 107*3).tolist()
glrlm_features = (np.arange(72,88) + 0).tolist() + (np.arange(72,88)  + 107).tolist() + (np.arange(72,88)  + 107*2).tolist() + (np.arange(72,88)  + 107*3).tolist()
ngtdm_features = (np.arange(88,93) + 0).tolist() + (np.arange(88,93)  + 107).tolist() + (np.arange(88,93)  + 107*2).tolist() + (np.arange(88,93) + 107*3).tolist()
gldm_features = (np.arange(93,107) + 0).tolist() + (np.arange(93,107)  + 107).tolist() + (np.arange(93,107)  + 107*2).tolist() + (np.arange(93,107) + 107*3).tolist()

feature_names = ['fos_features', '_3d_features', 'glcm_features', 'glszm_features', 'glrlm_features', 'ngtdm_features', 'gldm_features']
feature_indexes = [fos_features, _3d_features, glcm_features, glszm_features, glrlm_features, ngtdm_features, gldm_features]

In [ ]:
feature_number = 0

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 1

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 2

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 3

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 4

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_4 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 5

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_5 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 6

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_6 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)